In [ ]:

from torch import nn
# модель вручную копируем
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.layers = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1), # 32, 256, 256 
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2), # 32, 128, 128
            
            nn.Conv2d(32, 64, 3, padding=1), # 64, 128, 128 
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2), # 64, 64, 64
            
            nn.Conv2d(64, 128, 3, padding=1), # 128, 64, 64 
            nn.BatchNorm2d(128),
            nn.ReLU(),
            
            nn.Conv2d(128, 256, 3, padding=1), # 256, 64, 64 
            nn.BatchNorm2d(256),
            nn.ReLU(),
            
            nn.AvgPool2d(64),         # 256, 1, 1
            nn.Flatten(),             # 256
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 20)        # 20
        )

    def forward(self, x):
        return self.layers(x)

In [ ]:
import torch
# название нужное тоже ручками
model = Model()
model.load_state_dict(torch.load('./saved_models/w-05-04-25_00-00-53.pth'))

<All keys matched successfully>

In [3]:
import os
import pandas as pd
from PIL import Image
import torchvision.transforms.v2 as v2

def predict_and_save(model, img_folder, output_csv="predictions.csv"):

    # Проверка наличия папки
    if not os.path.exists(img_folder):
        raise FileNotFoundError(f"Папка {img_folder} не найдена")
    
    # Трансформации
    transform = v2.Compose([
        v2.Resize((256, 256)),
        v2.PILToTensor(),
        v2.ToDtype(torch.float32, scale=True),
        v2.Normalize([0.4674, 0.4498, 0.4123], [0.2686, 0.2641, 0.2690])
    ])
    
    # Сбор предсказаний
    results = []
    for img_name in os.listdir(img_folder):
        if img_name.lower().endswith('.jpg'):
            try:
                # Загрузка и преобразование
                img_path = os.path.join(img_folder, img_name)
                img = Image.open(img_path).convert('RGB')
                tensor = transform(img).unsqueeze(0).to(next(model.parameters()).device)
                
                # Предсказание
                with torch.no_grad():
                    pred = model(tensor).argmax().item()
                
                # Сохранение результата
                img_id = os.path.splitext(img_name)[0]
                results.append({"img_id": img_id, "target_feature": pred})
                
            except Exception as e:
                print(f"Ошибка при обработке {img_name}: {str(e)}")
    
    # Сохранение в CSV
    pd.DataFrame(results).to_csv(output_csv, index=False)
    print(f"Готово! Результаты сохранены в {output_csv}")

In [4]:
predict_and_save(model, './human_poses_data/img_test/')

Готово! Результаты сохранены в predictions.csv


In [27]:
# Проверка предсказаний на тестовых данных
test_input = torch.randn(1, 3, 256, 256)  # Тестовый тензор
with torch.no_grad():
    output = model(test_input)
    print("Raw output:", output)
    print("Predicted class:", torch.argmax(output).item())

Raw output: tensor([[ 2.2050, -2.0198,  0.8653,  1.4512, -1.0000,  1.0774,  1.1877, -4.7739,
         -1.2957,  1.0818,  0.2369,  0.1816, -0.5268,  0.1613, -0.4538, -5.7638,
          0.9688, -5.4395, -0.0962, -6.4506]])
Predicted class: 0
